참고 : 해당하는 달까지에 대한 데이터를 넣어주면, 최종적으로 다음달을 예측함!!!

- 현재 이 코드는 7월까지 있어서 8월(7월의 y_next)을 예측함 !!!!
- 우리가 9월데이터까지를 추가하면 10월꺼를 예측하는 것임!!! 

(맞지??)


타율, 방어율까지만 했고 창건오빠한테 승률 틀 받으면 그것도 만들어갈게유

In [1]:
# 기본
import os
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime as dt

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

C:\Users\KimMinyoung\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
data_dir='C:/Users/KimMinyoung/desktop/깃헙관련/빅콘git/data/total'
data_list = os.listdir(data_dir)
#print(len(data_list))
#print(data_list)

In [3]:
#년도별 저장
data_list_2016 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2016') ]
data_2016 = [pd.read_csv(os.path.join(data_dir,data_list_2016[x]),encoding='cp949') for x in range(len(data_list_2016))]
data_list_2017 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2017') ]
data_2017 = [pd.read_csv(os.path.join(data_dir,data_list_2017[x]),encoding='cp949') for x in range(len(data_list_2017))]
data_list_2018 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2018') ]
data_2018 = [pd.read_csv(os.path.join(data_dir,data_list_2018[x]),encoding='cp949') for x in range(len(data_list_2018))]
data_list_2019 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2019') ]
data_2019 = [pd.read_csv(os.path.join(data_dir,data_list_2019[x]),encoding='cp949') for x in range(len(data_list_2019))]
data_list_2020 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2020') ]
data_2020 = [pd.read_csv(os.path.join(data_dir,data_list_2020[x]),encoding='cp949') for x in range(len(data_list_2020))]
#print('2020년 데이터: \n', data_2020)

#항목별 저장
data_list_single_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인타자')]
data_single_hitter = [pd.read_csv(os.path.join(data_dir, data_list_single_hitter[x]), encoding='cp949') for x in range(len(data_list_single_hitter))]
data_list_single_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인투수')]
data_single_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_single_pitcher[x]), encoding='cp949') for x in range(len(data_list_single_pitcher))]
data_list_games = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('경기')]
data_games = [pd.read_csv(os.path.join(data_dir, data_list_games[x]), encoding='cp949') for x in range(len(data_list_games))]
data_list_player_enroll = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('등록선수')]
data_player_enroll = [pd.read_csv(os.path.join(data_dir, data_list_player_enroll[x]), encoding='cp949') for x in range(len(data_list_player_enroll))]
data_list_players = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('선수')]
data_players = [pd.read_csv(os.path.join(data_dir, data_list_players[x]), encoding='cp949') for x in range(len(data_list_players))]
data_list_teams = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀')]
data_teams = [pd.read_csv(os.path.join(data_dir, data_list_teams[x]), encoding='cp949') for x in range(len(data_list_teams))]
data_list_team_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀타자')]
data_team_hitter = [pd.read_csv(os.path.join(data_dir, data_list_team_hitter[x]), encoding='cp949') for x in range(len(data_list_team_hitter))]
data_list_team_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀투수')]
data_team_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_team_pitcher[x]), encoding='cp949') for x in range(len(data_list_team_pitcher))]
#print('팀투수 데이터:\n', data_team_pitcher)

# EDA

### 1. 투수

In [4]:
from sklearn.preprocessing import LabelEncoder

def team_pitcher_eda(year, team_name):
    year_index = year - 2016 
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_year = data_games[year_index]
    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_pitcher_year = data_team_pitcher[year_index]
    data_team_pitcher_year = pd.merge(data_team_pitcher_year, data_games_year, how='left', on=['G_ID'])

    data_team_pitcher_year_team = data_team_pitcher_year[data_team_pitcher_year.T_ID == team_name]
    df = data_team_pitcher_year_team
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'GDAY_DS'])
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    df['WLS'] = encoder.fit_transform(df['WLS']) #승패무 여부
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']

    #df['G_ID'] = pd.Series(dt)
    df['year']=df['G_ID'].str.slice(0,4).astype(int)
    df['month']=df['G_ID'].str.slice(4,6).astype(int)
    
    df["FIP"] = ((13*df.HR+3*(df.BB+df.HP-df.IB)-2*df.KK)/(df.INN2/3)) +3.2
    df["BABIP"] = (df.HIT-df.HR)/(df.AB-df.KK-df.HR+df.SF)
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']

    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('G_ID')

    # 경기당 방어율 column 생성
    df['ERA'] = df['ER'] * 27 / df['INN2']

    df = df.drop(columns=['T_ID'])
    df_year_team_name = df
    
    return df_year_team_name

### 2. 타자

In [5]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def team_hitter_eda(year, team_name):
    year_index = year - 2016 
    
    # team_hitter data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_year = data_games[year_index]
    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_hitter_year = data_team_hitter[year_index]
    data_team_hitter_year = pd.merge(data_team_hitter_year, data_games_year, how='left', on=['G_ID'])

    data_team_hitter_year_team = data_team_hitter_year[data_team_hitter_year.T_ID == team_name]
    df = data_team_hitter_year_team
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'G_ID'])
    
    df.reset_index(inplace=True, drop=True) #index 재정렬
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    # GDAY_DS => Datetime type
    df['GDAY_DS'] = df['GDAY_DS'].astype(str) + (df['HEADER_NO']+1).astype(str)
    df['year']=df['GDAY_DS'].str.slice(0,4).astype(int)
    df['month']=df['GDAY_DS'].str.slice(4,6).astype(int)
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']
    dt = []
    for i in df['GDAY_DS']:
        dt_ = datetime.strptime(i, '%Y%m%d%H')
        dt.append(dt_)
    df['GDAY_DS'] = pd.Series(dt)
    df['month']=df['GDAY_DS'].apply(lambda x: x.month)
    
    #장타율
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    #출루율
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']
    
    
    
    df = df.drop(columns=['T_ID'])
    #lags
    #lags=[1,4,6,12,30,60]
    #for lag in lags:
    #    df['AVG_lag_'+str(lag)]=df['AVG'].shift(lag).astype(np.float16)
    #SK Expanding window 추가
    #df['expanding_AVG_mean']=df['AVG'].transform(lambda x: x.expanding(2).mean().astype(np.float16))
    #rolling window
    #df['rolling_AVG_mean']=df['AVG'].transform(lambda x: x.rolling(window=7).mean().astype(np.float16))
    #trend
    
    # 경기당 타율 column 생성
    df['AVG'] = df['HIT'] / df['AB']
    
    df['avg_AVG'] = df['AVG'].mean()
    df['AVG_trend'] = (df['AVG'] - df['avg_AVG']).astype(np.float16)
    df.drop(['avg_AVG'],axis=1,inplace=True)
    
    # Drop Categorical feature for 시계열 => coint 과정에서 singular matrix 발생
    df = df.drop(columns=[ 'HOME_KEY'])
    

#     df = df.drop(columns=['T_ID', 'HEADER_NO', 'CG_CK', 'BK'])
#     df = df.drop(columns=['TB_SC', 'HR', 'SB', 'VS_T_ID', 'HOME_KEY', 'HOLD', 'INN2', 'BF', 'CS', 'SH', 'HP', 'GD', 'ERR' ,'ER'])
    df_year_team_name = df
    
    return df_year_team_name

### 3. 승률

In [6]:
def team_win_eda(year, team_name):
    
    
    
    return df_year_team_name

# Modeling

In [7]:
from sklearn.metrics import make_scorer

def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

def mse(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    sqr_val = difference ** 2
    
    score = sqr_val.mean()
    
    return score


def plot_train_test_pred_graph(trainset, testset, pred):
    plt.figure(figsize=(15,3))
    plt.plot(trainset.ERA, label='train')
    plt.plot(testset.ERA, label='test')
    plt.plot(testset.index, pred, label='prediction')
    plt.legend()
    plt.show()

In [8]:
# 얘는 쓰지 않음
def predict_columns(data_frame, x_cols, y_col):
    size = int(len(data_frame) * 0.7)
    train = data_frame[:size]
    test = data_frame[size:]
    
    reg_models = [LinearRegression(), Ridge(), Lasso(), ElasticNet(), GradientBoostingRegressor(), SVR(), XGBRegressor(), LGBMRegressor()]
    reg_model_names = ["LinearRegression", "Ridge", "Lasso", "ElasticNet", "GradientBoositng", " SupportVector", "XGBoost","lgbm"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols], train[y_col])
        pred = reg_models[i].predict(test[x_cols])
        print('{} : {}'.format(reg_model_names[i], mae(test[y_col], pred.astype(int))))
        #print(pred)

In [9]:
# 파라미터 지정

params_grid = {
    'num_leaves': [5,10,20,30],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [10,30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }

### 1. 투수

In [10]:
def team_pitcher_modeling(score, t):
    df_2016 = team_pitcher_eda(2016, t)
    df_2017 = team_pitcher_eda(2017, t)
    df_2018 = team_pitcher_eda(2018, t)
    df_2019 = team_pitcher_eda(2019, t)
    df_2020 = team_pitcher_eda(2020, t)
    
    df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
    df_all =df_all.reset_index()
    
    df_all['ym']=df_all['G_ID'].str.slice(0,6)

    a=df_all['ym'].unique()
    b=list(range(len(a)))
    list_C = [ x for x in zip(a,b) ]
    dict1=dict(list_C)

    df_all['dummy']=df_all['ym'].map(dict1)
    df_all['dummy+1']=df_all['dummy']+1

    y_dict=df_all.groupby(['dummy'])['ERA'].mean().to_dict()

    df_all['y_next']=df_all['dummy+1'].map(y_dict)
    df_all.drop(['dummy','dummy+1','ym'],axis=1,inplace=True)
    
    # 테스트용 데이터 뽑기
    final_test = df_all[df_all['y_next'].isnull()]; 
    
    df_all.dropna(inplace=True)#일단 드랍하고 진행
    
    df_all=df_all.drop(['G_ID'],axis=1)

    # 범주형 변수 지정
    cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
    df_all[cat_features] = df_all[cat_features].astype('category')

    # X,y Train split
    X = df_all.drop(columns = ['y_next'])
    y = df_all['y_next']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)
 
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]

    
    # 이부분 수정 필요
    lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
    lgb_grid = GridSearchCV(estimator=lgb,
                        param_grid=params_grid,
                        n_jobs=10,
                        verbose=3)
    lgb_grid.fit(X_train,y_train) 
    
    final_test = final_test.drop(['G_ID','y_next'],axis=1)
    final_test[cat_features] = final_test[cat_features].astype('category')
    
    final_predict = lgb_grid.predict(final_test)

    return final_predict

### 2. 타자

In [11]:
def team_hitter_modeling(score, t):
    df_2016 = team_hitter_eda(2016, t)
    df_2017 = team_hitter_eda(2017, t)
    df_2018 = team_hitter_eda(2018, t)
    df_2019 = team_hitter_eda(2019, t)
    df_2020 = team_hitter_eda(2020, t)
    
    df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
    df_all =df_all.reset_index()
    
    df_all['ym']=df_all['GDAY_DS'].apply(lambda x: str(x.year)+str(x.month))

    a=df_all['GDAY_DS'].apply(lambda x: str(x.year)+str(x.month)).unique()
    b=list(range(len(a)))
    list_C = [ x for x in zip(a,b) ]
    dict1=dict(list_C)

    df_all['dummy']=df_all['ym'].map(dict1)

    df_all['dummy+1']=df_all['dummy']+1

    y_dict=df_all.groupby(['dummy'])['AVG'].mean().to_dict()

    df_all['y_next']=df_all['dummy+1'].map(y_dict)

    df_all.drop(['dummy','dummy+1','ym'],axis=1,inplace=True)


    # 테스트용 데이터 뽑기
    final_test = df_all[df_all['y_next'].isnull()]; 
    
    df_all.dropna(inplace=True)#일단 드랍하고 진행
    
    df_all = df_all.drop(['GDAY_DS'],axis=1)

    # 범주형 변수 지정
    cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
    df_all[cat_features] = df_all[cat_features].astype('category')

    # X,y Train test val split
    X = df_all.drop(columns = ['y_next'])
    y = df_all['y_next']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)
 
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]

    
    # 이부분 수정 필요
    lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
    lgb_grid = GridSearchCV(estimator=lgb,
                        param_grid=params_grid,
                        n_jobs=10,
                        verbose=3)
    lgb_grid.fit(X_train,y_train) 
    
    print(final_test.columns)
    print(df_all.columns)
    final_test = final_test.drop(['GDAY_DS','y_next'],axis=1)
    final_test[cat_features] = final_test[cat_features].astype('category')
    
    final_predict = lgb_grid.predict(final_test)

    return final_predict

In [12]:
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']

## 팀별 최종 예측 방어율 (주어진 데이터 다음달)

In [ ]:
for t in team_list:
    print(t)
    print(team_pitcher_modeling("mae",t))

## 팀별 최종 예측 타율 (주어진 데이터 다음달)

In [ ]:
for t in team_list:
    print(t)
    print(team_hitter_modeling("mae",t))

## 팀별 최종 예측 승률 (주어진 데이터 다음달)